- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.

## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os
import pandas as pd
import re


In [19]:
import pandas as pd

In [2]:
chrome_options = webdriver.ChromeOptions()

# establacemos las preferencias que queremos
prefs = {
    "download.default_directory": "C:\\Users\\DELL\\Git\\Laboratorio_ETL\\datos_poblacion_provincia",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,
    "safebrowsing.enabled": True, # Desactiva las alertas de archivos peligrosos
    "plugins.always_open_pdf_externally": True,  
    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

url = "https://www.ine.es/dynt3/inebase/es/index.htm?type=pcaxis&path=/t20/e245/p08/&file=pcaxis&dh=0&capsel=2"

driver = webdriver.Chrome(options= chrome_options) # le decimos las opciones que hemos hecho previamente
chrome_options.add_experimental_option("prefs", prefs) #añade las opciones 


driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)

In [3]:
try:
    driver.find_element('xpath', '/html/body/div/main/section[2]/div[1]/div[1]/div[1]/ul/li/ul/li[3]/a').click()
except: 
    print('No encuentro el botón')


No encuentro el botón


In [4]:
driver.execute_script("window.scrollBy(0, 500);") 
sleep(3)

A por provincias

In [5]:
try:
    driver.find_element('xpath', '/html/body/div[1]/main/div[2]/ul/li[4]/ul/li[1]/a').click()
except:
    print('No encuentro provincia')
sleep(3)


In [6]:
# # Esta es otra forma de hacerlo
# edades = driver.find_elements("css selector", "#cri2")
# sleep(1)
# cant_edades = edades[0].find_elements(By.TAG_NAME, "option")
# print(len(cant_edades))
# sleep(1)
# for n in range(2, len(cant_edades) + 1):
#     #elemento = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cri1'))).click()
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri1"]/option[{n}]'))).click()
#     sleep(1)

# extranjeros = driver.find_elements("css selector", "#cri2")
# sleep(1)
# driver.find_element("xpath", "/html/body/div/main/form/ul/li[1]/ul/li[3]/div/fieldset/div[2]/button[2]/i").click()
# cant_extranjeros = extranjeros[0].find_elements(By.TAG_NAME, "option")
# sleep(1)
# for n in range(2, len(cant_extranjeros) + 1):
#     #elemento = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cri1'))).click()
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri2"]/option[{n}]'))).click()
#     sleep(1)

# generos = driver.find_elements("css selector", "#cri3")
# sleep(1)
# driver.find_element("xpath", "/html/body/div/main/form/ul/li[1]/ul/li[4]/div/fieldset/div[2]/button[2]/i").click()
# cant_generos = generos[0].find_elements(By.TAG_NAME, "option")
# sleep(1)
# for n in range(2, len(cant_generos) + 1):
#     #elemento = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#cri1'))).click()
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri3"]/option[{n}]'))).click()
#     sleep(1)

# anios = driver.find_elements("css selector", "#periodo")
# cant_anios = anios[0].find_elements(By.TAG_NAME, "option")
# sleep(1)
# for n in range(2, 5):
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="periodo"]/option[{n}]'))).click()
#     sleep(1)

In [7]:
try:
    driver.find_element('css selector', '#tg1 > div > fieldset > div.capaSelecTodosNinguno > button:nth-child(3) > i').click()
except:
    print('No encuentro sus edades')
sleep(3)

In [8]:
for i in range(2, 5):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri1"]/option[{i}]'))).click()
    sleep(1)
# intento que no coja todas pero sigue haciendolo...
# esto es para las edades

In [9]:
try:
    driver.find_element('css selector', '#tg2 > div > fieldset > div.capaSelecTodosNinguno > button:nth-child(3) > i').click()
except:
    print('No encuentro si son españoles o extranjeros')
sleep(3)

In [10]:
for i in range(2, 5):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri2"]/option[{i}]'))).click()
    sleep(1)

In [11]:
try:
    driver.find_element('css selector', '#tg3 > div > fieldset > div.capaSelecTodosNinguno > button:nth-child(3) > i').click()
except:
    print('No encuentro su sexo')
sleep(3)

In [12]:
for i in range(2, 4):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri3"]/option[{i}]'))).click()
    sleep(1)

In [13]:
# for n in range(2,4):
#     WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cri1"]/option[{n}]'))).click()
#     sleep(1)

# no traer de regreso

In [14]:
try:
    driver.find_element('css selector', '#caja_periodo > div > fieldset > div.capaSelecTodosNinguno > button:nth-child(3) > i').click()
except:
    print('No encuentro el blanco')
sleep(3)

In [15]:
for n in range(2,5):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="periodo"]/option[{n}]'))).click()
    sleep(1)

In [16]:
driver.execute_script("window.scrollBy(0, 500);") 

In [17]:
driver.find_element("css selector", "#botonConsulSele").click()

In [18]:
sleep(1)
driver.find_element("xpath", "/html/body/div[1]/main/ul/li/div/div/form[2]/button/i").click()
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="thickBoxINEfrm"]')))
sleep(1)
driver.switch_to.frame(iframe)
sleep(1)
driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
sleep(1)
driver.switch_to.default_content()
sleep(1)
# aqui acaba todo